# fastcore_meta

## Delegates

### Import libraries

In [ ]:
#|default_exp fastcoremeta

In [ ]:
#|export
from fastcore.meta import *
from fastcore.test import *
import inspect
from pprint import pprint

In [ ]:
from nbdev.showdoc import *
from IPython.display import IFrame

### Reading the doc

#### how-sig: access signature, quick doc and full doc

In [ ]:
#|echo: false
pprint(inspect.signature(delegates))
pprint(inspect.getdoc(delegates))

<Signature (to: function = None, keep=False, but: list = None, verbose=True)>
'Decorator: replace `**kwargs` in signature with params from `to`'


In [ ]:
#|echo: false
IFrame("https://fastcore.fast.ai/meta.html#delegates", 1200,500)

#### how-assert: use `assert`
more [examples](https://realpython.com/python-testing/)

```python
assert foo(c=1, a=1) == 7
```

#### how-delegates: Which params passed to `f` from `to` in source?

- only keyword params (with default values) are delegated to `foo` from `baz`
- params (both positional and keyword) from `foo` suffice to fill positional params of `baz`


#### how-deletates: when params in `f` conflict with kwargs of `to`

In [ ]:
def baz(a, b=2, c=3): return a + b + c

@delegates(baz)
def foo1(d, a, **kwargs): # both b=2, c=3 are kept
    return d + baz(a, **kwargs)

@delegates(baz)
def foo2(c, a, **kwargs): # c in foo2 replaces c=3 in baz, only b=2 is kept
    return c + baz(a, **kwargs)

inspect.signature(foo1), inspect.signature(foo2) 

(<Signature (d, a, *, b=2, c=3)>, <Signature (c, a, *, b=2)>)

#### how-sig: `test_sig`

In [ ]:
test_sig(foo2, '(c, a, *, b=2)')

#### how-delegates: on classmethod and instant method

In [ ]:
class _T():
    @classmethod
    def foo(cls, a=1, b=2):
        pass
    
    @classmethod
    @delegates(foo)
    def bar(cls, c=3, **kwargs):
        pass

In [ ]:
t = _T()
inspect.signature(t.bar)

<Signature (c=3, *, a=1, b=2)>

In [ ]:
class _T():
    def foo(cls, a=1, b=2):
        pass
    
    @delegates(foo)
    def bar(cls, c=3, **kwargs):
        pass

In [ ]:
t = _T()
inspect.signature(t.bar)

<Signature (c=3, *, a=1, b=2)>

#### how-delegates: on super class or `__init__` func in fact

In [ ]:
class BaseFoo:
    def __init__(self, e, c=2): pass

class Foo(BaseFoo):
    def __init__(self, a, b=1, **kwargs): super().__init__(**kwargs)

test_sig(Foo, '(a, b=1, **kwargs)')

In [ ]:
class BaseFoo:
    def __init__(self, e, c=2): pass

@delegates()# since no argument was passsed here we delegate to the superclass
class Foo(BaseFoo):
    def __init__(self, a, b=1, **kwargs): super().__init__(**kwargs)

test_sig(Foo, '(a, b=1, *, c=2)')

#### how-delegates: on other class

In [ ]:
class BaseFoo:
    def __init__(self, e, c=2): pprint(locals())

class OtherFoo:
    def __init__(self, d, f=1): pprint(locals())

@delegates(OtherFoo)# since no argument was passsed here we delegate to the superclass
class Foo(BaseFoo):
    def __init__(self, a, b=1, **kwargs): 
        pprint(locals())
        super().__init__(**kwargs)

# inspect.signature(Foo)
test_sig(Foo, '(a, b=1, *, f=1)')

In [ ]:
try:        
    f = Foo(1,1,f=1)
except TypeError as e:
    print(e)

{'__class__': <class '__main__.Foo'>,
 'a': 1,
 'b': 1,
 'kwargs': {'f': 1},
 'self': <__main__.Foo object>}
__init__() got an unexpected keyword argument 'f'


#### how-delegates: before its time

In [ ]:
def low(a:int, b:int=1): pass
def mid(c:int, d:int=1, **kwargs): low(c, **kwargs) + d
def high(e:int, f:int=1, **kwargs): mid(e, **kwargs) + f

In [ ]:
print("low: ")
pprint(inspect.signature(low))
print("mid: ")
pprint(inspect.signature(mid))
print("high: ")
pprint(inspect.signature(high))

low: 
<Signature (a: int, b: int = 1)>
mid: 
<Signature (c: int, d: int = 1, **kwargs)>
high: 
<Signature (e: int, f: int = 1, **kwargs)>


#### how-delegates: under its reign

In [ ]:
def low(a:int, b:int=1): pass
@delegates(low)
def mid(c:int, d:int=1, **kwargs): low(c, **kwargs) + d
@delegates(mid)
def high(e:int, f:int=1, **kwargs): mid(e, **kwargs) + f

In [ ]:
print("low: ")
pprint(inspect.signature(low))
print("mid: ")
pprint(inspect.signature(mid))
print("high: ")
pprint(inspect.signature(high))

low: 
<Signature (a: int, b: int = 1)>
mid: 
<Signature (c: int, d: int = 1, *, b: int = 1)>
high: 
<Signature (e: int, f: int = 1, *, d: int = 1, b: int = 1)>


#### how-show_doc: `show_doc` can even informs you which kwargs are passed to other funcs

#### how-delegates: use `keep=True` and `but`

In [ ]:
@delegates(mid, keep=True, but=['b'])
def high(e:int, f:int=1, **kwargs): mid(e, **kwargs) + f
pprint(inspect.signature(high))

<Signature (e: int, f: int = 1, *, d: int = 1, **kwargs)>


#### how-delegates: without `@`

Using `delegates??` to get its source code

```python
def delegates(to=None, keep=False, but=None):
    "Decorator: replace `**kwargs` in signature with params from `to`"
    if but is None: but = []
    def _f(f):
        if to is None: to_f,from_f = f.__base__.__init__,f.__init__
        else:          to_f,from_f = to.__init__ if isinstance(to,type) else to,f
        from_f = getattr(from_f,'__func__',from_f)
        to_f = getattr(to_f,'__func__',to_f)
        if hasattr(from_f,'__delwrap__'): return f
        sig = inspect.signature(from_f)
        sigd = dict(sig.parameters)
        k = sigd.pop('kwargs')
        s2 = {k:v for k,v in inspect.signature(to_f).parameters.items()
              if v.default != inspect.Parameter.empty and k not in sigd and k not in but}
        sigd.update(s2)
        if keep: sigd['kwargs'] = k
        else: from_f.__delwrap__ = to_f
        from_f.__signature__ = sig.replace(parameters=sigd.values())
        return f
    return _f
```

In [ ]:
def low(a:int, b:int=1): return a + b
def mid(c:int, d:int=1, **kwargs): return low(c, **kwargs) + d

In [ ]:
print(mid(1))
inspect.signature(mid)

3


<Signature (c: int, d: int = 1, **kwargs)>

In [ ]:
_f = delegates(low)
mid = _f(mid)
print(mid(1))

inspect.signature(mid)

3


<Signature (c: int, d: int = 1, *, b: int = 1)>

`@delegates(low)` is equivalent to the first two lines in cell above

### The anatomy of `delegates`

#### Examples

Function example

In [ ]:
def low(a:int, b:int=1): return a + b
def mid(c:int, d:int=1, **kwargs): return low(c, **kwargs) + d
def high(e:int, f:int=1, **kwargs): return mid(e, **kwargs) + f

Class example

In [ ]:
class BaseFoo:
    def __init__(self, e, c=2): pass

class OtherFoo:
    def __init__(self, d, f=1): pass

class Foo(BaseFoo):
    def __init__(self, a, b=1, **kwargs): super().__init__(**kwargs)

```python
def delegates(to=None, keep=False, but=None):
    "Decorator: replace `**kwargs` in signature with params from `to`"
    if but is None: but = []
    def _f(f):
        if to is None: to_f,from_f = f.__base__.__init__,f.__init__
        else:          to_f,from_f = to.__init__ if isinstance(to,type) else to,f
        from_f = getattr(from_f,'__func__',from_f)
        to_f = getattr(to_f,'__func__',to_f)
        if hasattr(from_f,'__delwrap__'): return f
        sig = inspect.signature(from_f)
        sigd = dict(sig.parameters)
        k = sigd.pop('kwargs')
        s2 = {k:v for k,v in inspect.signature(to_f).parameters.items()
              if v.default != inspect.Parameter.empty and k not in sigd and k not in but}
        sigd.update(s2)
        if keep: sigd['kwargs'] = k
        else: from_f.__delwrap__ = to_f
        from_f.__signature__ = sig.replace(parameters=sigd.values())
        return f
    return _f
```

#### Params and inputs

```python
def delegates(to=None, keep=False, but=None):
    "Decorator: replace `**kwargs` in signature with params from `to`"
    if but is None: but = []
```

In [ ]:
# def delegates(to=None, keep=False, but=None):
#     "Decorator: replace `**kwargs` in signature with params from `to`"
to=mid     # `to` is the func to delegate its kwargs with default values to `f` func
keep=False # whether to keep `**kwargs` inside `f` signature or not
but=None   # which default kwargs to remove from `f` signature
if but is None: but = [] # if not None, but should be a list type

#### Prepare `to` and `f`

```python
...
    def _f(f):
        if to is None: to_f,from_f = f.__base__.__init__,f.__init__
        else:          to_f,from_f = to.__init__ if isinstance(to,type) else to,f
        from_f = getattr(from_f,'__func__',from_f)
        to_f = getattr(to_f,'__func__',to_f)
        ....
```

What can `to` and `f` be?
- function
- class method, instant method
- class
- type?

When dealing with class and superclass

In [ ]:
to = None
f = Foo
if to is None: to_f,from_f = f.__base__.__init__,f.__init__ # use delegates on class
to_f, from_f

(<function __main__.BaseFoo.__init__(self, e, c=2)>,
 <function __main__.Foo.__init__(self, a, b=1, **kwargs)>)

#### how-attr: get attr by name or else: `getattr`

In [ ]:
from_f = getattr(from_f,'__func__',from_f)
to_f = getattr(to_f,'__func__',to_f); to_f, from_f

(<function __main__.BaseFoo.__init__(self, e, c=2)>,
 <function __main__.Foo.__init__(self, a, b=1, **kwargs)>)

#### how-code: assign values to a tuple `else: to_f,from_f = to.__init__ if isinstance(to,type) else to,f`

In [ ]:
to = 1
f = 34
if True: to_f,from_f = 3 if to == 1 else to,f 
print(to_f, from_f, f)

3 34 34


In [ ]:
to = 1
f = 34
if True: 
    if to == 1: 
        to_f = 3
        from_f = f
    else: 
        to_f = to
        from_f = f
print(to_f, from_f, f)

3 34 34


#### how-class: check object is a class `isinstance(to, type) == True`
how-type: learn more of [type](https://hyp.is/nean3hlREe2QFO-_fpoP7g/stackoverflow.com/questions/100003/what-are-metaclasses-in-python)

In [ ]:
to = OtherFoo
f = Foo
print(isinstance(to,type))
# if isinstance(to,type): to_f,from_f = to.__init__ 
if to is None: to_f,from_f = f.__base__.__init__,f.__init__ # use delegates on class
else:          to_f,from_f = to.__init__ if isinstance(to,type) else to,f 
to_f, from_f

True


(<function __main__.OtherFoo.__init__(self, d, f=1)>, __main__.Foo)

In [ ]:
from_f = getattr(from_f,'__func__',from_f)
to_f = getattr(to_f,'__func__',to_f);  to_f, from_f

(<function __main__.OtherFoo.__init__(self, d, f=1)>, __main__.Foo)

Working with function

In [ ]:
# def _f(f):
to = low
f = mid
if to is None: to_f,from_f = f.__base__.__init__,f.__init__ # use delegates on class
else:          to_f,from_f = to.__init__ if isinstance(to,type) else to,f 
                                         # secret use of delegates on type
                                         # finally delegates on functions
to_f, from_f

(<function __main__.low(a: int, b: int = 1)>,
 <function __main__.mid(c: int, d: int = 1, **kwargs)>)

In [ ]:
from_f = getattr(from_f,'__func__',from_f)
to_f = getattr(to_f,'__func__',to_f);  to_f, from_f

(<function __main__.low(a: int, b: int = 1)>,
 <function __main__.mid(c: int, d: int = 1, **kwargs)>)

#### Make sure no delegates apply to `f` without `**kwargs`

see when `__delwrap__` attr is set below.

```python
if hasattr(from_f,'__delwrap__'): return f # don't do it more than once
```

#### How-attr: whether a function has an attr or not: `hasattr`

In [ ]:
hasattr(from_f,'__delwrap__') == False

True

In [ ]:
pprint(inspect.signature(from_f))
hasattr(from_f,'__signature__') == False

<Signature (c: int, d: int = 1, **kwargs)>


True

Function Example

In [ ]:
def low(a, b:int=1): pass
@delegates(low)
def mid(c, d:int=1, **kwargs): pass
inspect.signature(mid)

<Signature (c, d: int = 1, *, b: int = 1)>

In [ ]:
mid.__delwrap__, hasattr(mid, '__delwrap__')

(<function __main__.low(a, b: int = 1)>, True)

Class Example

In [ ]:
class BaseFoo:
    def __init__(self, e, c=2): pass

@delegates()# since no argument was passsed here we delegate to the superclass
class Foo(BaseFoo):
    def __init__(self, a, b=1, **kwargs): super().__init__(**kwargs)
        
inspect.signature(Foo)

<Signature (a, b=1, *, c=2)>

In [ ]:
Foo.__init__.__delwrap__, hasattr(Foo.__init__, '__delwrap__')

(<function __main__.BaseFoo.__init__(self, e, c=2)>, True)

#### apply `fastcore.delegates` multiple times to a func when `keep=True`

You could do it with no `@`, but no useful usecases for it.

In [ ]:
def low(a, b:int=1): pass
def mid(c, d:int=1, **kwargs): pass
inspect.signature(mid)

<Signature (c, d: int = 1, **kwargs)>

In [ ]:
mid1 = delegates(low, keep=True)(mid)
test_sig(mid, '(c, d: int = 1, *, b: int = 1, **kwargs)')
def low1(a1, b1:int=1): pass
mid2 = delegates(low1, keep=True)(mid)
test_sig(mid, '(c, d: int = 1, *, b: int = 1, b1: int = 1, **kwargs)')

#### Remove `**kwargs` and add kwargs with default

```python
...
    def _f(f):
        ...
        
        sig = inspect.signature(from_f)
        sigd = dict(sig.parameters)
        k = sigd.pop('kwargs')
        s2 = {k:v for k,v in inspect.signature(to_f).parameters.items()
              if v.default != inspect.Parameter.empty and k not in sigd and k not in but}
        sigd.update(s2)

        ...
        return f
    return _f
```

#### how-dict: pop out an item by key name `**kwargs`

In [ ]:
sig = inspect.signature(from_f) # get signature
print(sig, "\n")

sigd = dict(sig.parameters) # turn signature into a dict
pprint(sigd)
k = sigd.pop('kwargs') # take **kwargs parameter out of the dict and keep it

print("")
pprint(k) # **kwargs parameter
pprint(sig) # unchanged
pprint(sigd) # changed

(c: int, d: int = 1, **kwargs) 

{'c': <Parameter "c: int">,
 'd': <Parameter "d: int = 1">,
 'kwargs': <Parameter "**kwargs">}

<Parameter "**kwargs">
<Signature (c: int, d: int = 1, **kwargs)>
{'c': <Parameter "c: int">, 'd': <Parameter "d: int = 1">}


#### how-try-except: When `f` has no `**kwargs`, KeyError occurs

In [ ]:
def low(a, b:int=1): pass

try:
    @delegates(low)
    def mid(c, d:int=1): pass
except KeyError as e:
    print(f"when mid has no **kwargs parameter in its signature, fastcore.delegates will throw an KeyError: {e}")

when mid has no **kwargs parameter in its signature, fastcore.delegates will throw an KeyError: 'kwargs'


#### get default kwargs from `to`

Those kwargs must meet 3 conditions
- have default values or `!= inspect.Parameter.empty`
- not conflict with parameters in `f`
- not in the list of `but`

In [ ]:
def low(a, b:int=1, c:int=2, e:int=1): pass
def mid(c, d:int=1, **kwargs): pass
to_f = low
sig = inspect.signature(mid)
pprint(sig.parameters)
sigd = dict(sig.parameters) # Note: sig.parameters
k = sigd.pop("kwargs")
but = ['b']
s2 = {k:v for k,v in inspect.signature(to_f).parameters.items()
      if v.default != inspect.Parameter.empty and k not in sigd and k not in but}; 
s2 # is empty dict because c is part of `mid` too, b is in the `but` list

mappingproxy(OrderedDict([('c', <Parameter "c">),
                          ('d', <Parameter "d: int = 1">),
                          ('kwargs', <Parameter "**kwargs">)]))


{'e': <Parameter "e: int = 1">}

#### how-inspect.Signature: replace parameter's kind to `Parameter.KEYWORD_ONLY`

In [ ]:
s2 = {name: param.replace(kind=inspect.Parameter.KEYWORD_ONLY) for name, param in s2.items()} 
s2['e'].kind   
    

<_ParameterKind.KEYWORD_ONLY: 3>

#### how-inspect.Parameter: empty default value with `inspect.Parameter.empty`

```python
if v.default != inspect.Parameter.empty
```

#### how-dict: update: add additional items into a dict

In [ ]:
sigd.update(s2); sigd

{'c': <Parameter "c">,
 'd': <Parameter "d: int = 1">,
 'e': <Parameter "e: int = 1">}

#### If `keep=True`, then add `**kwargs` back to `from_f`'s signature

```python
...
    def _f(f):
        ...
        if keep: sigd['kwargs'] = k
        else: from_f.__delwrap__ = to_f
        from_f.__signature__ = sig.replace(parameters=sigd.values())
        return f
    return _f
```

In [ ]:
def low(a, b=1): pass
def mid(c, d, **kwargs): pass
hasattr(mid, '__signature__')

False

In [ ]:
def low(a, b=1): pass
@delegates(low)
def mid(c, d, **kwargs): pass
hasattr(mid, '__signature__')

True

#### If `keep=False`, do `from_f.setattr("__delwrap__", to_f)`

#### how-attr: two ways of setting attr: `setattr` or `from_f.__signature__= ...`

In [ ]:
try: 
    from_f.__delwrap__
except AttributeError as e:
    print(e)
    
to_f, from_f

'function' object has no attribute '__delwrap__'


(<function __main__.low(a, b: int = 1, c: int = 2, e: int = 1)>,
 <function __main__.mid(c: int, d: int = 1, **kwargs)>)

In [ ]:
keep = False
if keep == False: from_f.__delwrap__ = to_f # another way to set attr
from_f.__delwrap__

<function __main__.low(a, b: int = 1, c: int = 2, e: int = 1)>

#### how-attr: set and delete an attr

In [ ]:
def tst(): pass
tst.__setattr__("__delwrap__", "a")
print(tst.__delwrap__)
tst.__delwrap__ = "b"
print(tst.__delwrap__)
tst.__delattr__("__delwrap__")
try: 
    print(tst.__delwrap__)
except: 
    print("no more")

a
b
no more


#### Replace the new args dict with the old args in the signature of `from_f`

In [ ]:
sigd

{'c': <Parameter "c">,
 'd': <Parameter "d: int = 1">,
 'e': <Parameter "e: int = 1">}

#### how-dict: get all items (key and value) of a dict

In [ ]:
sigd.items()

dict_items([('c', <Parameter "c">), ('d', <Parameter "d: int = 1">), ('e', <Parameter "e: int = 1">)])

#### how-dict: get only values of a dict

In [ ]:
sigd.values()

dict_values([<Parameter "c">, <Parameter "d: int = 1">, <Parameter "e: int = 1">])

#### how-inspect: getdoc and Signature.replace

In [ ]:
pprint(inspect.getdoc(sig.replace))

('Creates a customized copy of the Signature.\n'
 "Pass 'parameters' and/or 'return_annotation' arguments\n"
 'to override them in the new copy.')


In [ ]:
try:
    from_f.__signature__
except AttributeError as e:
    print(e)

from_f

'function' object has no attribute '__signature__'


<function __main__.mid(c: int, d: int = 1, **kwargs)>

In [ ]:
from_f.__signature__ = sig.replace(parameters=sigd.values())
pprint(from_f.__signature__)

<Signature (c, d: int = 1, *, e: int = 1)>


In [ ]:
from_f

<function __main__.mid(c, d: int = 1, *, e: int = 1)>

In [ ]:
# return _f

### problem-delegates: What may go wrong when `keep=True` and func `low` has no `**kwargs`

In [ ]:
def low(a:int,   
        b:int=1, 
       ):
    print("the locals of low: ")
    pprint(locals())
    return a + b

@delegates(low)
def mid(c:int,   
        d:int=1, 
        **kwargs 
       ):
    print("the locals of mid: ")
    pprint(locals())
    return low(c, **kwargs) + d

@delegates(mid)
def high(e:int,   
         f:int=1, 
         **kwargs 
        ):
    print("the locals of high: ")
    pprint(locals())
    return mid(e, **kwargs) + f

In [ ]:
mid(c=2,d=1,b=1) # use shift + tab

the locals of mid: 
{'c': 2, 'd': 1, 'kwargs': {'b': 1}}
the locals of low: 
{'a': 2, 'b': 1}


4

In [ ]:
high(e=2,f=1,d=1,b=1) 

the locals of high: 
{'e': 2, 'f': 1, 'kwargs': {'b': 1, 'd': 1}}
the locals of mid: 
{'c': 2, 'd': 1, 'kwargs': {'b': 1}}
the locals of low: 
{'a': 2, 'b': 1}


5

When `keep=True`, we can assume the user of `delegates` would like to make `**kwargs` visible and use additional keyword args at some point. 

But when `low` has no `**kwargs`, and any additional kwargs (e.g. `g=1`) introduced will cause `low`to throw a TypeError. See example below.

In [ ]:
def low(a:int,   
        b:int=1, 
       ):
    print("the locals of low: ")
    pprint(locals())
    return a + b
@delegates(low, keep=True)
def mid(c:int,   
        d:int=1, 
        **kwargs 
       ):
    print("the locals of mid: ")
    pprint(locals())
    return low(c, **kwargs) + d
@delegates(mid, keep=True)
def high(e:int,   
         f:int=1, 
         **kwargs 
        ):
    print("the locals of high: ")
    pprint(locals())
    return mid(e, **kwargs) + f

In [ ]:
try: 
    high(e=2,f=1,d=1,b=1,g=1) # shift + tab to see **kwargs available
except TypeError as e: 
    print(e)

the locals of high: 
{'e': 2, 'f': 1, 'kwargs': {'b': 1, 'd': 1, 'g': 1}}
the locals of mid: 
{'c': 2, 'd': 1, 'kwargs': {'b': 1, 'g': 1}}
low() got an unexpected keyword argument 'g'


After inserting `**kwargs` into function `low`, `g=1` is happily accepted by `low` in `kwargs`.

In [ ]:
def low(a:int, b:int=2, **kwargs): 
    print("the locals of low: ")
    pprint(locals())
    return a + b
test_eq(high(e=2,f=1,d=1,b=1,g=1),5) 

the locals of high: 
{'e': 2, 'f': 1, 'kwargs': {'b': 1, 'd': 1, 'g': 1}}
the locals of mid: 
{'c': 2, 'd': 1, 'kwargs': {'b': 1, 'g': 1}}
the locals of low: 
{'a': 2, 'b': 1, 'kwargs': {'g': 1}}


Wouldn't it be nicer if we can remind or warn the `delegates` user that `low` with no `**kwargs` will cause error when you set `keep=True`?

### fastai makes no mistake like this

See below the [one](https://github.com/fastai/fastai/blob/825c27624e4f6612fc0ce87769d9238d0ab96bab/fastai/torch_core.py#L40) of a few [examples](https://github.com/fastai/fastai/search?q=delegates+%22keep%3DTrue%22) when fastai uses `keep=True` I have found on August 11 2022.

how-delegates: correct way of using `keep=True`: pay attention to the first line and last line of the code block below

```python
@delegates(plt.subplots, keep=True)
def subplots(
    nrows:int=1, # Number of rows in returned axes grid
    ncols:int=1, # Number of columns in returned axes grid
    figsize:tuple=None, # Width, height in inches of the returned figure 
    imsize:int=3, # Size (in inches) of images that will be displayed in the returned figure
    suptitle:str=None, # Title to be set to returned figure
    **kwargs
) -> (plt.Figure, plt.Axes): # Returns both fig and ax as a tuple 
    "Returns a figure and set of subplots to display images of `imsize` inches"
    if figsize is None: 
        h=nrows*imsize if suptitle is None or imsize>2 else nrows*imsize+0.6 #https://github.com/matplotlib/matplotlib/issues/5355
        figsize=(ncols*imsize, h)
    fig,ax = plt.subplots(nrows, ncols, figsize=figsize, **kwargs) # must have **kwargs here
```

### tinker-delegates: warn user the lowest level `to` has no `**kwargs`, don't pass expected params

#### how-try-except: replace the combination of `if else` + `hasattr`

#### how-attr: every func has an attr `__name__`

In [ ]:
#|export
def delegates_v1(to=None, # delegates `to` to replace `**kwargs` with its own params
              keep=False, # keep `**kwargs` in the signature or not
              but:list=None): # leave out certain params from the signature
    """
        Decorator: replace `**kwargs` in signature with params from `to`. If `keep=True`and `to` has no `**kwargs` in its signature, a warning of TypeError will be generated.
    """
    if but is None: but = []
    def _f(f):
        if to is None: to_f,from_f = f.__base__.__init__,f.__init__
        else:          to_f,from_f = to.__init__ if isinstance(to,type) else to,f
        from_f = getattr(from_f,'__func__',from_f)
        to_f = getattr(to_f,'__func__',to_f)
        if hasattr(from_f,'__delwrap__'): return f
        sig = inspect.signature(from_f)
        sigd = dict(sig.parameters)
        k = sigd.pop('kwargs')
            
        s2 = {k:v for k,v in inspect.signature(to_f).parameters.items()
              if v.default != inspect.Parameter.empty and k not in sigd and k not in but}
        sigd.update(s2)

        """
        if to_f has no attr 'lowest_func_no_kwargs', 
        then this to_f must be the first or the lowest function;
        if this to_f has no **kwargs as parameter, then we set the value of attr 'lowest_func_no_kwargs'
        to be to_f itself in string format
        if this to_f has **kwargs as parameter, then set the value of the attr as None
        """
        sigt = inspect.signature(to_f)  # get signature of mid
        sigtd = dict(sigt.parameters)  # make signature a dict
        try:
            from_f.__setattr__("lowest_func_no_kwargs", to_f.lowest_func_no_kwargs)
        except:
            if sigtd.get('kwargs') == None: 
                to_f.__setattr__("lowest_func_no_kwargs", f'{to_f.__name__}')
            else:
                to_f.__setattr__("lowest_func_no_kwargs", None)
            from_f.__setattr__("lowest_func_no_kwargs", to_f.lowest_func_no_kwargs)
        
        if keep: 
            if to_f.lowest_func_no_kwargs != None:  # check lowest_no_kwargs
                print(f"Warning: adding extra kwargs to {from_f.__name__} will cause TypeError to {to_f.lowest_func_no_kwargs}\n")
            sigd['kwargs'] = k
        else: from_f.__delwrap__ = to_f
        from_f.__signature__ = sig.replace(parameters=sigd.values())
        return f
    return _f

#### examples of v_1

In [ ]:
def low(a:int, b:int=1): pass

@delegates_v1(low, keep=True)
def mid(c:int, d:int=1, **kwargs): pass

In [ ]:
@delegates_v1(mid, keep=True)
def high(e:int, f:int=1, **kwargs): pass

### problem-delegates: params from `to` act as KEYWORD_ONLY but its type says otherwirse

In [ ]:
import fastcore.meta as fm

In [ ]:
def low(a, b:int=1): pass
@fm.delegates(low, keep=True)
def mid(c, d:int=1, **kwargs): pass

In [ ]:
inspect.signature(mid)

<Signature (c, d: int = 1, *, b: int = 1, **kwargs)>

The signature above indicate that `b` from `low` is `POSITIONAL_OR_KEYWORD` kind. However, if you run `b` as so, it causes a TypeError like below.

In [ ]:
try: 
    mid(1,1,1)
except TypeError as e:
    print(e)

mid() takes from 1 to 2 positional arguments but 3 were given


#### how-inspect: check param kind

`b` in practice is a `KEYWORD_ONLY` param but when its type checked it is not.

In [ ]:
{p.name: p.kind for p in inspect.signature(mid).parameters.values()}

{'c': <_ParameterKind.POSITIONAL_OR_KEYWORD: 1>,
 'd': <_ParameterKind.POSITIONAL_OR_KEYWORD: 1>,
 'b': <_ParameterKind.KEYWORD_ONLY: 3>,
 'kwargs': <_ParameterKind.VAR_KEYWORD: 4>}

### tinker-delegates: printing a warning message about param kind

Let's make `Core.delegates_v2` to print a message for users to use params from `to` as Keyword only params

In [ ]:
s = {'a':1, 'b':2}
[i for i in s.keys()]

['a', 'b']

In [ ]:
#|export
def delegates_v2(to=None, # delegates `to` to replace `**kwargs` with its own params
              keep=False, # keep `**kwargs` in the signature or not
              but:list=None): # leave out certain params from the signature
    "Compared with v1, delegates_v2 func add a feature to remind users that params from `to` are keyword-only params for `f`."

    if but is None: but = []
    def _f(f):
        if to is None: to_f,from_f = f.__base__.__init__,f.__init__
        else:          to_f,from_f = to.__init__ if isinstance(to,type) else to,f
        from_f = getattr(from_f,'__func__',from_f)
        to_f = getattr(to_f,'__func__',to_f)
        if hasattr(from_f,'__delwrap__'): return f
        sig = inspect.signature(from_f)
        sigd = dict(sig.parameters)
        k = sigd.pop('kwargs')
            
        s2 = {k:v for k,v in inspect.signature(to_f).parameters.items()
              if v.default != inspect.Parameter.empty and k not in sigd and k not in but}
        sigd.update(s2)
        """
        put s2's keys into a list
        print a warning message: from_f has the s2's keys as Keyword Only params
        """
        keyword_only_list = [i for i in s2.keys()]
        print(f"{from_f.__name__} has {keyword_only_list} as keyword only params, not to be used as positional.")

        """
        if to_f has no attr 'lowest_func_no_kwargs', 
        then this to_f must be the first or the lowest function;
        if this to_f has no **kwargs as parameter, then we set the value of attr 'lowest_func_no_kwargs'
        to be to_f itself in string format
        if this to_f has **kwargs as parameter, then set the value of the attr as None
        """
        sigt = inspect.signature(to_f)  # get signature of mid
        sigtd = dict(sigt.parameters)  # make signature a dict
        try:
            from_f.__setattr__("lowest_func_no_kwargs", to_f.lowest_func_no_kwargs)
        except:
            if sigtd.get('kwargs') == None: 
                to_f.__setattr__("lowest_func_no_kwargs", f'{to_f.__name__}')
            else:
                to_f.__setattr__("lowest_func_no_kwargs", None)
            from_f.__setattr__("lowest_func_no_kwargs", to_f.lowest_func_no_kwargs)
        
        if keep: 
            if to_f.lowest_func_no_kwargs != None:  # check lowest_no_kwargs
                print(f"Warning: adding extra kwargs to {from_f.__name__} will cause TypeError to {to_f.lowest_func_no_kwargs}\n")
            sigd['kwargs'] = k
        else: from_f.__delwrap__ = to_f
        from_f.__signature__ = sig.replace(parameters=sigd.values())
        return f
    return _f

#### delegates_v2 examples

In [ ]:
def low(a, b:int=1): pass
@delegates_v2(low, keep=True)
def mid(c, d:int=1, **kwargs): pass
@delegates_v2(mid, keep=True)
def high(e, f:int=1, **kwargs): pass

mid has ['b'] as keyword only params, not to be used as positional.

high has ['d', 'b'] as keyword only params, not to be used as positional.



In [ ]:
{p.name: p.kind for p in inspect.signature(mid).parameters.values()}

{'c': <_ParameterKind.POSITIONAL_OR_KEYWORD: 1>,
 'd': <_ParameterKind.POSITIONAL_OR_KEYWORD: 1>,
 'b': <_ParameterKind.POSITIONAL_OR_KEYWORD: 1>,
 'kwargs': <_ParameterKind.VAR_KEYWORD: 4>}

In [ ]:
try: 
    mid(1,1,1)
except TypeError as e:
    print(e)

mid() takes from 1 to 2 positional arguments but 3 were given


### tinker-delegates: change parameter type to `Parameter.KEYWORD_ONLY`

In [ ]:
#|export
def delegates_v3(to=None, # delegates `to` to replace `**kwargs` with its own params
              keep=False, # keep `**kwargs` in the signature or not
              but:list=None): # leave out certain params from the signature
    "Compared with v1, delegates_v2 func add a feature to remind users that params from `to` are keyword-only params for `f`."

    if but is None: but = []
    def _f(f):
        if to is None: to_f,from_f = f.__base__.__init__,f.__init__
        else:          to_f,from_f = to.__init__ if isinstance(to,type) else to,f
        from_f = getattr(from_f,'__func__',from_f)
        to_f = getattr(to_f,'__func__',to_f)
        if hasattr(from_f,'__delwrap__'): return f
        sig = inspect.signature(from_f)
        sigd = dict(sig.parameters)
        k = sigd.pop('kwargs')
            
        s2 = {k:v for k,v in inspect.signature(to_f).parameters.items()
              if v.default != inspect.Parameter.empty and k not in sigd and k not in but}
        """
        Change type from POSITIONAL_OR_KEYWORD to KEYWORD_ONLY for all parameters in s2
        """
        s2 = {name: param.replace(kind=inspect.Parameter.KEYWORD_ONLY) for name, param in s2.items()} 
        
        sigd.update(s2)
        """
        put s2's keys into a list
        print a warning message: from_f has the s2's keys as Keyword Only params
        """
        keyword_only_list = [i for i in s2.keys()]
        print(f"{from_f.__name__} has {keyword_only_list} as keyword only params, not to be used as positional.")

        """
        if to_f has no attr 'lowest_func_no_kwargs', 
        then this to_f must be the first or the lowest function;
        if this to_f has no **kwargs as parameter, then we set the value of attr 'lowest_func_no_kwargs'
        to be to_f itself in string format
        if this to_f has **kwargs as parameter, then set the value of the attr as None
        """
        sigt = inspect.signature(to_f)  # get signature of mid
        sigtd = dict(sigt.parameters)  # make signature a dict
        try:
            from_f.__setattr__("lowest_func_no_kwargs", to_f.lowest_func_no_kwargs)
        except:
            if sigtd.get('kwargs') == None: 
                to_f.__setattr__("lowest_func_no_kwargs", f'{to_f.__name__}')
            else:
                to_f.__setattr__("lowest_func_no_kwargs", None)
            from_f.__setattr__("lowest_func_no_kwargs", to_f.lowest_func_no_kwargs)
        
        if keep: 
            if to_f.lowest_func_no_kwargs != None:  # check lowest_no_kwargs
                print(f"Warning: adding extra kwargs to {from_f.__name__} will cause TypeError to {to_f.lowest_func_no_kwargs}\n")
            sigd['kwargs'] = k
        else: from_f.__delwrap__ = to_f
        from_f.__signature__ = sig.replace(parameters=sigd.values())
        return f
    return _f

### contribute-delegates: [PR](https://app.reviewnb.com/fastai/fastcore/pull/459/) accepted on August 11 2022

By adding a single line of code below the line where `s2` occurs.
```python
# Change type from POSITIONAL_OR_KEYWORD to KEYWORD_ONLY for all parameters in s2
s2 = {name: param.replace(kind=inspect.Parameter.KEYWORD_ONLY) for name, param in s2.items()} 
```

With Jeremy's [help](https://github.com/fastai/fastcore/pull/459#discussion_r942706390), I have shortened the code by simply modifying the line started with `s2=`

In [ ]:
def delegates(to=None, keep=False, but=None):
    "Decorator: replace `**kwargs` in signature with params from `to`"
    if but is None: but = []
    def _f(f):
        if to is None: to_f,from_f = f.__base__.__init__,f.__init__
        else:          to_f,from_f = to.__init__ if isinstance(to,type) else to,f
        from_f = getattr(from_f,'__func__',from_f)
        to_f = getattr(to_f,'__func__',to_f)
        if hasattr(from_f,'__delwrap__'): return f
        sig = inspect.signature(from_f)
        sigd = dict(sig.parameters)
        k = sigd.pop('kwargs')
        s2 = {k:v.replace(kind=inspect.Parameter.KEYWORD_ONLY) for k,v in inspect.signature(to_f).parameters.items()
              if v.default != inspect.Parameter.empty and k not in sigd and k not in but}
        sigd.update(s2)
        if keep: sigd['kwargs'] = k
        else: from_f.__delwrap__ = to_f
        from_f.__signature__ = sig.replace(parameters=sigd.values())
        return f
    return _f

In [ ]:
def low(a, b:int=1): pass
@delegates(low, keep=True)
def mid(c, d:int=1, **kwargs): pass
@delegates(mid, keep=True)
def high(e, f:int=1, **kwargs): pass

In [ ]:
{p.name: p.kind for p in inspect.signature(mid).parameters.values()}

{'c': <_ParameterKind.POSITIONAL_OR_KEYWORD: 1>,
 'd': <_ParameterKind.POSITIONAL_OR_KEYWORD: 1>,
 'b': <_ParameterKind.KEYWORD_ONLY: 3>,
 'kwargs': <_ParameterKind.VAR_KEYWORD: 4>}

In [ ]:
inspect.signature(mid)

<Signature (c, d: int = 1, *, b: int = 1, **kwargs)>

## _mk_param

how-parameter: How to make a parameter

### how-import: What does `from fastcore.meta import *` mean

`from fastcore.meta import *` won't give you everything inside `meta` module but a selective group contained in a list `__all__`, to use the unselected ones, you can: 
```python
import fastcore.met as fm
```
and use `fm. + tab` to check out everything.

In [ ]:
!head /Users/Natsume/mambaforge/lib/python3.9/site-packages/fastcore/meta.py

head: /Users/Natsume/mambaforge/lib/python3.9/site-packages/fastcore/meta.py: No such file or directory


### `_mk_param` examples

In [ ]:
from fastcore.meta import _mk_param

In [ ]:
_mk_param('b'), _mk_param('b', 1)

(<Parameter "b=None">, <Parameter "b=1">)

### `_mk_param` source

In [ ]:
def _mk_param(n,d=None): return inspect.Parameter(n, inspect.Parameter.KEYWORD_ONLY, default=d)

```python
def _mk_param(n,d=None): return inspect.Parameter(n, inspect.Parameter.KEYWORD_ONLY, default=d)
```

### `_mk_param` anatomy

#### how-inspect: get signature

In [ ]:
inspect.signature(inspect.Parameter)

<Signature (name, kind, *, default, annotation)>

#### how-inspect: getdoc

In [ ]:
pprint(inspect.getdoc(inspect.Parameter))

('Represents a parameter in a function signature.\n'
 '\n'
 'Has the following public attributes:\n'
 '\n'
 '* name : str\n'
 '    The name of the parameter as a string.\n'
 '* default : object\n'
 '    The default value for the parameter if specified.  If the\n'
 '    parameter has no default value, this attribute is set to\n'
 '    `Parameter.empty`.\n'
 '* annotation\n'
 '    The annotation for the parameter if specified.  If the\n'
 '    parameter has no annotation, this attribute is set to\n'
 '    `Parameter.empty`.\n'
 '* kind : str\n'
 '    Describes how argument values are bound to the parameter.\n'
 '    Possible values: `Parameter.POSITIONAL_ONLY`,\n'
 '    `Parameter.POSITIONAL_OR_KEYWORD`, `Parameter.VAR_POSITIONAL`,\n'
 '    `Parameter.KEYWORD_ONLY`, `Parameter.VAR_KEYWORD`.')


#### how-inspect: create Parameter

In [ ]:
p = inspect.Parameter("b", inspect.Parameter.KEYWORD_ONLY, default=None)
type(p), p

(inspect.Parameter, <Parameter "b=None">)

### 

## 

## use_kwargs_dict
how-kwargs: replace `**kwargs` with a dict of params

### `use_kwargs_dic` source

In [ ]:
def use_kwargs_dict(keep=False, **kwargs):
    "Decorator: replace `**kwargs` in signature with `names` params"
    def _f(f):
        sig = inspect.signature(f)
        sigd = dict(sig.parameters)
        k = sigd.pop('kwargs')
        s2 = {n:_mk_param(n,d) for n,d in kwargs.items() if n not in sigd}
        sigd.update(s2)
        if keep: sigd['kwargs'] = k
        f.__signature__ = sig.replace(parameters=sigd.values())
        return f
    return _f

#### how-code: put `for in` and `if else` in a single line

```python
s2 = {n:_mk_param(n,d) for n,d in kwargs.items() if n not in sigd}
```

#### most attributes like `__signature__`, `__func__` are added by decorators

In [ ]:
def foo(a, b=1, **kwargs): pass
hasattr(foo, '__signature__')

False

In [ ]:
@use_kwargs_dict(y=1,z=None)
def foo(a, b=1, **kwargs): pass
hasattr(foo, '__signature__')

True

### `use_kwargs_dict` examples

Replace all **kwargs with named arguments like so:

In [ ]:
@use_kwargs_dict(y=1,z=None)
def foo(a, b=1, **kwargs): pass

test_sig(foo, '(a, b=1, *, y=1, z=None)')

Add named arguments, but optionally keep **kwargs by setting keep=True:

In [ ]:
@use_kwargs_dict(y=1,z=None, keep=True)
def foo(a, b=1, **kwargs): pass

test_sig(foo, '(a, b=1, *, y=1, z=None, **kwargs)')

## 

## use_kwargs
how-kwargs: to replace `**kwargs` with a list of param names. `use_kwargs`

### `use_kwargs` source

In [ ]:
def use_kwargs(names, keep=False):
    "Decorator: replace `**kwargs` in signature with `names` params"
    def _f(f):
        sig = inspect.signature(f)
        sigd = dict(sig.parameters)
        k = sigd.pop('kwargs')
        s2 = {n:_mk_param(n) for n in names if n not in sigd}
        sigd.update(s2)
        if keep: sigd['kwargs'] = k
        f.__signature__ = sig.replace(parameters=sigd.values())
        return f
    return _f

### `use_kwargs` examples

use_kwargs is different than use_kwargs_dict as it only replaces **kwargs with named parameters without any default values:

In [ ]:
@use_kwargs(['y', 'z'])
def foo(a, b=1, **kwargs): pass

test_sig(foo, '(a, b=1, *, y=None, z=None)')
inspect.signature(foo)

<Signature (a, b=1, *, y=None, z=None)>

In [ ]:
try:
    foo(1,1,1)
except TypeError as e:
    print(e)
assert foo(1,1,y=1) == foo(1,1,y=1,z=None)

foo() takes from 1 to 2 positional arguments but 3 were given


You may optionally keep the `**kwargs` argument in your signature by setting `keep=True`:

In [ ]:
@use_kwargs(['y', 'z'], keep=True)
def foo(a, *args, b=1, **kwargs): pprint(locals())
test_sig(foo, '(a, *args, b=1, y=None, z=None, **kwargs)')

### how-code: how `*args` in signature behave?

In [ ]:
foo(1,[1],y=1)

{'a': 1, 'args': ([1],), 'b': 1, 'kwargs': {'y': 1}}


In [ ]:
foo(1,[1,1],y=1)

{'a': 1, 'args': ([1, 1],), 'b': 1, 'kwargs': {'y': 1}}


In [ ]:
foo(1,1,1,y=1)

{'a': 1, 'args': (1, 1), 'b': 1, 'kwargs': {'y': 1}}


In [ ]:
foo(1,1,1,b=1,y=1)

{'a': 1, 'args': (1, 1), 'b': 1, 'kwargs': {'y': 1}}


## 

## test_sig
how-test: signature

### `test_sig` source

In [ ]:
def test_sig(f, b):
    "Test the signature of an object"
    test_eq(str(inspect.signature(f)), b)

### `test_sig` examples

In [ ]:
def func_1(h,i,j): pass
def func_2(h,i=3, j=[5,6]): pass

class T:
    def __init__(self, a, b): pass

test_sig(func_1, '(h, i, j)')
test_sig(func_2, '(h, i=3, j=[5, 6])')
test_sig(T, '(a, b)')

### `test_sig` anatomy

In [ ]:
inspect.signature(T)

<Signature (a, b)>

#### how-str: turn signature into string

In [ ]:
str(inspect.signature(T))

'(a, b)'

In [ ]:
inspect.signature(func_2)

<Signature (h, i=3, j=[5, 6])>

In [ ]:
str(inspect.signature(func_2))

'(h, i=3, j=[5, 6])'

In [ ]:
test_eq(str(inspect.signature(func_2)), '(h, i=3, j=[5, 6])')

In [ ]:
assert str(inspect.signature(func_2)) == '(h, i=3, j=[5, 6])' 

## 

## `_rm_self`
how-inspect/how-self: remove `self` in signature

### `_rm_self` source

In [ ]:
def _rm_self(sig):
    sigd = dict(sig.parameters)
    sigd.pop('self')
    return sig.replace(parameters=sigd.values())

### `_rm_self` examples

In [ ]:
class Base:
    def __init__(self, a, b:int=1, **kwargs): pass
    def test(self, c, d): pass

In [ ]:
inspect.signature(Base.__init__)

<Signature (self, a, b: int = 1, **kwargs)>

In [ ]:
_rm_self(inspect.signature(Base.__init__))

<Signature (a, b: int = 1, **kwargs)>

In [ ]:
inspect.signature(Base.test)

<Signature (self, c, d)>

In [ ]:
_rm_self(inspect.signature(Base.test))

<Signature (c, d)>

### `_rm_self` anatomy

In [ ]:
sig = inspect.signature(Base.__init__)
sig

<Signature (self, a, b: int = 1, **kwargs)>

In [ ]:
sigd = dict(sig.parameters)
sigd

{'self': <Parameter "self">,
 'a': <Parameter "a">,
 'b': <Parameter "b: int = 1">,
 'kwargs': <Parameter "**kwargs">}

In [ ]:
sigd.pop('self')
sigd

{'a': <Parameter "a">,
 'b': <Parameter "b: int = 1">,
 'kwargs': <Parameter "**kwargs">}

In [ ]:
sigd.values()

dict_values([<Parameter "a">, <Parameter "b: int = 1">, <Parameter "**kwargs">])

In [ ]:
sig.replace(parameters=sigd.values())

<Signature (a, b: int = 1, **kwargs)>

## 

## FixSigMeta

A metaclass that fixes the signature on classes that override `__new__`

When you inherit from a class that defines `__new__`, or a metaclass that defines `__call__`, the signature of your `__init__` method is obfuscated such that tab completion no longer works. FixSigMeta fixes this issue and restores signatures.

To understand what `FixSigMeta` does, it is useful to inspect an object's signature. You can inspect the signature of an object with `inspect.signature`:

Interesting! `FixSigMeta??` won't give us the source code, neither can `inspect.getsource` on it.

### `FixSigMeta` examples

#### examples not really working effectively

In [ ]:
class T:
    def __init__(self, a, b, c): pass
    
inspect.signature(T)

<Signature (a, b, c)>

This corresponds to tab completion working in the normal way:

`T() + shift + tab` can provide the following info on the function 
```
Init signature: T(a, b, c)
Docstring:      <no docstring>
Type:           type
```

What is the signature of `T.__init__`

However, when you inherhit from a class that defines `__new__` or a metaclass that defines `__call__` this obfuscates the signature by overriding your class with the signature of `__new__`, which prevents tab completion from displaying useful information:

In [ ]:
class Foo:
    def __new__(self, **args): pass
    
class Bar(Foo):
    def __init__(self, d, e, f): pass

In [ ]:
pprint(inspect.signature(Foo))
pprint(inspect.signature(Foo.__new__))

<Signature (**args)>
<Signature (self, **args)>


In [ ]:
pprint(inspect.signature(Bar))
pprint(inspect.signature(Bar.__new__))
pprint(inspect.signature(Bar.__init__))

<Signature (d, e, f)>
<Signature (self, **args)>
<Signature (self, d, e, f)>


Finally, the signature and tab completion can be restored by inheriting from the metaclass `FixSigMeta` as shown below:

#### how-FixSigMeta: when inherit a class override `__new__`


In [ ]:
#|export
class FixSigMeta(type):
    "A metaclass that fixes the signature on classes that override `__new__` of `type`"
    def __new__(cls, name, bases, dict):
        print("============================\nrunning metaclass FixSigMeta.__new__\n")
        print(f"local variables are:\n")
        pprint(locals())
        
        # create new obj using type.__new__
        res = super().__new__(cls, name, bases, dict)
        print(f"\nUsing type.__new__ to create a class instance of type: {res}")
        
        pprint(f"{res.__name__}'s type should and is: {type(res)}")
        print(f"\n{res.__name__}'s inheritance tree: {res.__mro__}")
        print(f"In other words, {res.__name__}'s bases are: {res.__bases__}\n")
        print(f"\ndoes {res.__name__} have attr `__signature__`:{hasattr(res, '__signature__')}")
        pprint(inspect.signature(res))
        
        
        print(f"if {res.__name__} has its own __init__, then update with __init__'s signature")
        if res.__init__ is not object.__init__: res.__signature__ = _rm_self(inspect.signature(res.__init__))
        if hasattr(res, '__signature__'):
            print(f"\nnow, {res.__name__}.__signature__ becomes:{res.__signature__}")
        
        print("\nend of metaclass FixSigMeta.__new__\n============================")
        return res # and the object is passed on

In [ ]:
class Foo:
    def __new__(cls, *args): # to create an instance of class Foo
        print("============================\nRunning Foo.__new__")
        
        print(f"local variables inside Foo.__new__:\n")
        pprint(locals())
        print(f"cls.__class__ is: {cls.__class__}")
        
        print("\nwhat is super():\n")
        pprint(super())
        res = super().__new__(cls) # equivalent to object.__new__(cls), 
        # it is to create an object of class Bar whose superclass is Foo
        print(f"super().__new__ created: {res}\n")

        return res
    
# FixSigMeta is called here to create Bar class, not to create instance of Bar
class Bar(Foo, metaclass=FixSigMeta): 
    # running Foo.__new__ here
    print(f"=================================We are in the context of Bar: \n")
    
    def __init__(self, d, e, f): 
        print("\n==================================\n we are inside Bar.__init__")
        print(f"local variables available to Bar.__init__:\n")
        pprint(locals())

=================================We are in the context of Bar: 

running metaclass FixSigMeta.__new__

local variables are:

{'__class__': <class '__main__.FixSigMeta'>,
 'bases': (<class '__main__.Foo'>,),
 'cls': <class '__main__.FixSigMeta'>,
 'dict': {'__init__': <function Bar.__init__>,
          '__module__': '__main__',
          '__qualname__': 'Bar'},
 'name': 'Bar'}

Using type.__new__ to create a class instance of type: <class '__main__.Bar'>
"Bar's type should and is: <class '__main__.FixSigMeta'>"

Bar's inheritance tree: (<class '__main__.Bar'>, <class '__main__.Foo'>, <class 'object'>)
In other words, Bar's bases are: (<class '__main__.Foo'>,)


does Bar have attr `__signature__`:False
<Signature (d, e, f)>
if Bar has its own __init__, then update with __init__'s signature

now, Bar.__signature__ becomes:(d, e, f)

end of metaclass FixSigMeta.__new__


In [ ]:
test_eq(isinstance(Bar, FixSigMeta), True) # FixSigMeta created Bar
test_eq(isinstance(Bar, Foo), False) 
test_eq(issubclass(Bar, Foo), True) # Bar inherited from Foo
test_eq(issubclass(Foo, object), True)

In [ ]:
# When creating an instance of Bar
# It will not run FixSigMeta.__new__, but run FixSigMeta.__call__ (not specified) 
# as a result, inside Bar it inherits and runs Foo.__new__, 
# and then inherits and overrides Foo.__init__ with Bar.__init__
b = Bar(1,2,3) 

Running Foo.__new__
local variables inside Foo.__new__:

{'__class__': <class '__main__.Foo'>,
 'args': (1, 2, 3),
 'cls': <class '__main__.Bar'>}
cls.__class__ is: <class '__main__.FixSigMeta'>

what is super():

<super: <class 'Foo'>, <Bar object>>
super().__new__ created: <__main__.Bar object>


 we are inside Bar.__init__
local variables available to Bar.__init__:

{'d': 1, 'e': 2, 'f': 3, 'self': <__main__.Bar object>}


#### how-FixSigMeta: when inherit a metaclass to override `__call__`

If you need to define a metaclass that overrides `__call__` (as done in `PrePostInitMeta`), you need to inherit from `FixSigMeta` instead of `type` when constructing the metaclass to preserve the signature in `__init__`. Be careful not to override `__new__` when doing this:

In [ ]:
class TestMeta(FixSigMeta): # making a subclass of FixSigMeta, which overrides __call__
    # __new__ comes from FixSigMeta
    def __call__(cls, *args, **kwargs): pass
    

In [ ]:
class T(metaclass=TestMeta):
    def __init__(self, a, b): pass
    

running metaclass FixSigMeta.__new__

local variables are:

{'__class__': <class '__main__.FixSigMeta'>,
 'bases': (),
 'cls': <class '__main__.TestMeta'>,
 'dict': {'__init__': <function T.__init__>,
          '__module__': '__main__',
          '__qualname__': 'T'},
 'name': 'T'}

Using type.__new__ to create a class instance of type: <class '__main__.T'>
"T's type should and is: <class '__main__.TestMeta'>"

T's inheritance tree: (<class '__main__.T'>, <class 'object'>)
In other words, T's bases are: (<class 'object'>,)


does T have attr `__signature__`:False
<Signature (*args, **kwargs)>
if T has its own __init__, then update with __init__'s signature

now, T.__signature__ becomes:(a, b)

end of metaclass FixSigMeta.__new__


In [ ]:
test_sig(T, '(a, b)')

On the other hand, if you fail to inherit from `FixSigMeta` when inheriting from a metaclass that overrides `__call__`, your signature will reflect that of `__call__` instead (which is often undesirable):

In [ ]:
class GenericMeta(type): # create a metaclass from type and overrides both __new__ and __call_
    "A boilerplate metaclass that doesn't do anything for testing."
    
    def __new__(cls, name, bases, dict):
        print("============================\nrunning metaclass GenericMeta.__new__\n")
        print(f"local variables available to GenericMeta.__new__:\n")
        pprint(locals())
        
        res = super().__new__(cls, name, bases, dict) # run type.__new__()
        print(f"\ntype.__new__ created: {res}")
        pprint(f"{res.__name__}'s type is {type(res)}")
        print(f"\n{res.__name__}'s inheritance tree: {res.__mro__}")
        print(f"\ndoes {res.__name__} have attr `__signature__`:{hasattr(res, '__signature__')}")
        pprint(inspect.signature(res))
        print(f"{res.__name__}'s bases are: {res.__bases__}\n")
        
        return res
    
    def __call__(cls, *args, **kwargs): print("==========================\nrunning GenericMeta.__call__")

In [ ]:
class T2(metaclass=GenericMeta):
    def __init__(self, a, b): pass

running metaclass GenericMeta.__new__

local variables available to GenericMeta.__new__:

{'__class__': <class '__main__.GenericMeta'>,
 'bases': (),
 'cls': <class '__main__.GenericMeta'>,
 'dict': {'__init__': <function T2.__init__>,
          '__module__': '__main__',
          '__qualname__': 'T2'},
 'name': 'T2'}

type.__new__ created: <class '__main__.T2'>
"T2's type is <class '__main__.GenericMeta'>"

T2's inheritance tree: (<class '__main__.T2'>, <class 'object'>)

does T2 have attr `__signature__`:False
<Signature (*args, **kwargs)>
T2's bases are: (<class 'object'>,)



In [ ]:
T2() # to instantiate an object of T2 class, GenericMeta.__call__ is called under the scene

running GenericMeta.__call__


In [ ]:
class GenericMeta(FixSigMeta): # make GenericMeta a subclass of metaclass FixSigMeta
    "A boilerplate metaclass that doesn't do anything for testing."
    def __new__(cls, name, bases, dict):
        print("============================\nrunning metaclass GenericMeta.__new__\n")
        print(f"local variables available to GenericMeta.__new__:\n")
        pprint(locals())
        
        res = super().__new__(cls, name, bases, dict) # run type.__new__()
        print(f"\ntype.__new__ created: {res}")
        pprint(f"{res.__name__}'s type is {type(res)}")
        print(f"\n{res.__name__}'s inheritance tree: {res.__mro__}")
        print(f"\ndoes {res.__name__} have attr `__signature__`:{hasattr(res, '__signature__')}")
        pprint(inspect.signature(res))
        print(f"{res.__name__}'s bases are: {res.__bases__}\n")
        
        return res
    
    
    def __call__(cls, *args, **kwargs): print("==========================\nrunning GenericMeta.__call__")

class T2(metaclass=GenericMeta): # create T2 class as an instance of GenericMeta class
    def __init__(self, a, b): pass

test_sig(T2, '(a, b)')

running metaclass GenericMeta.__new__

local variables available to GenericMeta.__new__:

{'__class__': <class '__main__.GenericMeta'>,
 'bases': (),
 'cls': <class '__main__.GenericMeta'>,
 'dict': {'__init__': <function T2.__init__>,
          '__module__': '__main__',
          '__qualname__': 'T2'},
 'name': 'T2'}
running metaclass FixSigMeta.__new__

local variables are:

{'__class__': <class '__main__.FixSigMeta'>,
 'bases': (),
 'cls': <class '__main__.GenericMeta'>,
 'dict': {'__init__': <function T2.__init__>,
          '__module__': '__main__',
          '__qualname__': 'T2'},
 'name': 'T2'}

Using type.__new__ to create a class instance of type: <class '__main__.T2'>
"T2's type should and is: <class '__main__.GenericMeta'>"

T2's inheritance tree: (<class '__main__.T2'>, <class 'object'>)
In other words, T2's bases are: (<class 'object'>,)


does T2 have attr `__signature__`:False
<Signature (*args, **kwargs)>
if T2 has its own __init__, then update with __init__'s signature

In [ ]:
FixSigMeta??

## 

## PrePostInitMeta

### Relations between PrePostInitMeta and FixSigMeta

In [ ]:
#|export
class PrePostInitMeta(FixSigMeta): # make a subclass of metaclass FixSigMeta, is now a metaclass
    "A metaclass that calls optional `__pre_init__` and `__post_init__` methods"
    
    print(f"==================\nRunning inside PrePostInitMeta\n")
    print(f"PrePostInitMeta will inherit __new__ and __init__ from FixSigMeta")
    
    # override the metaclass FixSigMeta.__call__
    def __call__(cls, *args, **kwargs):
        print(f"==================\nRunning PrePostInitMeta.__call__\n")
        print(f"local variables:")
        pprint(locals())
        
        # let the cls to create an object instance based on cls, and assign obj to res
        res = cls.__new__(cls) # res is a class
        print(f"\nTask 1 running res = {cls.__name__}.__new__(cls) to create an obj instance of {cls.__name__}: {res}")
        
        if type(res)==cls: # if res is an instance of cls
            
            print(f"\nTask 2: \nif {res.__class__.__name__} object is an instance of {cls.__name__} and \
{res.__class__.__name__} has '__pre_init__' attr, then run {res.__class__.__name__}.__pre_init__ on the object.")
            # if the instance class has __pre__init__, then run this func
            if hasattr(res,'__pre_init__'): res.__pre_init__(*args,**kwargs)
                
            print(f"\nTask 3: \nif {res.__class__.__name__} object is an instance of {cls.__name__}, \
then run {res.__class__.__name__}.__init__ on the object.")
            res.__init__(*args,**kwargs) # call res.__init__
            
       
            print(f"\nTask 4: \nif {res.__class__.__name__} object is an instance of {cls.__name__} and \
{res.__class__.__name__} has '__post_init__' attr, then run {res.__class__.__name__}.__post_init__ on the object.")
            if hasattr(res,'__post_init__'): res.__post_init__(*args,**kwargs)

            print(f"\nfinally, return the initialized object of class {res.__class__.__name__}")
            print(f"====================end of PrePostInitMeta.__call__")
            return res # return obj
    print(f"====================end of PrePostInitMeta class")

Running inside PrePostInitMeta

PrePostInitMeta will inherit __new__ and __init__ from FixSigMeta
====================end of PrePostInitMeta class


In [ ]:
type(PrePostInitMeta) == PrePostInitMeta.__class__ == type

True

In [ ]:
PrePostInitMeta.__bases__ # so will inherit __new__, __init__ from FixSigMeta

(__main__.FixSigMeta,)

In [ ]:
PrePostInitMeta.__mro__

(__main__.PrePostInitMeta, __main__.FixSigMeta, type, object)

In [ ]:
FixSigMeta.__mro__

(__main__.FixSigMeta, type, object)

### `PrePostInitMeta` examples

A metaclass that calls optional `__pre_init__` and `__post_init__` methods `__pre_init__` and `__post_init__` are useful for initializing variables or performing tasks prior to or after `__init__` being called, respectively. Fore example:

In [ ]:
# When creating an instance class by metaclass PrePostInitMeta, 
# PrePostInitMeta.__new__ is called, but PrePostInitMeta inherit __new__ from FixSigMeta
# so FixSigMeta.__new__ is actually called below

# _T is inherited from object and will use object.__new__ and object.__init__ 
# _T needs to create its own instance objects

# _T builts its own 3 instance methods which all related to __init__

# todo: export my debugging version of FixSigMeta to the library
# todo: maybe the first problem of delegates do need attention
class _T(metaclass=PrePostInitMeta):
    print("\n===============Running inside _T\n")
    def __pre_init__(self):  
        print("====Running inside _T.__pre_init__\n")
        self.a  = 0; 
    def __init__(self,b=0):  
        self.b = self.a + 1; assert self.b==1
        print("====Running inside _T.__init__\n")
    def __post_init__(self): 
        self.c = self.b + 2; assert self.c==3
        print("====Running inside _T.__post_init__\n")


===============Running inside _T

running metaclass FixSigMeta.__new__

local variables are:

{'__class__': <class '__main__.FixSigMeta'>,
 'bases': (),
 'cls': <class '__main__.PrePostInitMeta'>,
 'dict': {'__init__': <function _T.__init__>,
          '__module__': '__main__',
          '__post_init__': <function _T.__post_init__>,
          '__pre_init__': <function _T.__pre_init__>,
          '__qualname__': '_T'},
 'name': '_T'}

Using type.__new__ to create a class instance of type: <class '__main__._T'>
"_T's type should and is: <class '__main__.PrePostInitMeta'>"

_T's inheritance tree: (<class '__main__._T'>, <class 'object'>)
In other words, _T's bases are: (<class 'object'>,)


does _T have attr `__signature__`:False
<Signature (*args, **kwargs)>
if _T has its own __init__, then update with __init__'s signature

now, _T.__signature__ becomes:(b=0)

end of metaclass FixSigMeta.__new__


In [ ]:
# when _T is instantiated, _T's metaclass'__call__ (i.e., PrePostInitMeta.__call__) is called 
# _T.__class__.__call__ in general will use _T.__new__ and _T.__init__ 
# to create and initialize an instance object of _T.

# PrePostInitMeta.__call__: has a particular way of creating and initializing an object
# using _T.__new__, _T.__init__, and _T.__pre_init__, _T.__post_init__ 

t = _T() # only instantiate an object, to do so, PrePostInitMeta.__call__ is called

Running PrePostInitMeta.__call__

local variables:
{'args': (), 'cls': <class '__main__._T'>, 'kwargs': {}}

Task 1 running res = _T.__new__(cls) to create an obj instance of _T: <__main__._T object>

Task 2: 
if _T object is an instance of _T and _T has '__pre_init__' attr, then run _T.__pre_init__ on the object.
====Running inside _T.__pre_init__


Task 3: 
if _T object is an instance of _T, then run _T.__init__ on the object.
====Running inside _T.__init__


Task 4: 
if _T object is an instance of _T and _T has '__post_init__' attr, then run _T.__post_init__ on the object.
====Running inside _T.__post_init__


finally, return the initialized object of class _T
====================end of PrePostInitMeta.__call__


In [ ]:
test_eq(t.a, 0) # set with __pre_init__
test_eq(t.b, 1) # set with __init__
test_eq(t.c, 3) # set with __post_init__

how-prepostinitmeta: One use for PrePostInitMeta is avoiding the __super__().__init__() boilerplate associated with subclassing, such as used in AutoInit.

## 

## `AutoInit`

how-super: a good [answer](https://stackoverflow.com/questions/576169/understanding-python-super-with-init-methods) on `super` and a [great guide](https://rhettinger.wordpress.com/2011/05/26/super-considered-super/)? and see `PrePostInitMeta` and `AutoInit` make it even better! 

### `AutoInit` source

In [ ]:
#|export
# AutoInit is a class created by metaclass PrePostInitMeta, not type
# AutoInit inherit from object, not PrePostInitMeta
# when AutoInit makes an object instance, 
# the PrePostInitMeta.__call__ makes use of __pre_init__, __init__, __post_init__ of AutoInit
# if AutoInit has them

# but right now, only PrePostInitMeta.__new__ is called to create a class instance of itself
# AntoInit is not a metaclass, but a class instance of PrePostInitMeta
class AutoInit(metaclass=PrePostInitMeta):
    "Same as `object`, but no need for subclasses to call `super().__init__`"
    
    print(f"\n=========Running inside AutoInit class\n")
    
    def __pre_init__(self, *args, **kwargs): 
        print(f"\n====Running inside AutoInit.__pre_init__: \n")
        
        super().__init__(*args, **kwargs)
        print(f"Now, object.__init__ is called to initialize, so that subclasses of AutoInit \
        not need to run this line of code.")


=========Running inside AutoInit class

running metaclass FixSigMeta.__new__

local variables are:

{'__class__': <class '__main__.FixSigMeta'>,
 'bases': (),
 'cls': <class '__main__.PrePostInitMeta'>,
 'dict': {'__classcell__': <cell at 0x110bbbaf0: empty>,
          '__doc__': 'Same as `object`, but no need for subclasses to call '
                     '`super().__init__`',
          '__module__': '__main__',
          '__pre_init__': <function AutoInit.__pre_init__>,
          '__qualname__': 'AutoInit'},
 'name': 'AutoInit'}

Using type.__new__ to create a class instance of type: <class '__main__.AutoInit'>
"AutoInit's type should and is: <class '__main__.PrePostInitMeta'>"

AutoInit's inheritance tree: (<class '__main__.AutoInit'>, <class 'object'>)
In other words, AutoInit's bases are: (<class 'object'>,)


does AutoInit have attr `__signature__`:False
<Signature (*args, **kwargs)>
if AutoInit has its own __init__, then update with __init__'s signature

end of metaclass FixSigM

### `AutoInit` examples

how-prepostinitmeta: in fastai [code](https://github.com/fastai/fastai/search?q=PrePostInitMeta)

how-autoinit: use `AutoInit` in real life. This is normally used as a mixin, eg:

In [ ]:
class TestParent():
    def __init__(self): self.h = 10
        
class TestChild(AutoInit, TestParent): # how AutoInit used as mixin
    print(f"============Running inside TestChild class:\n")
    def __init__(self): 
        self.k = self.h + 2 # No More worry about super().__init__ 
        print(f"====running TestChild.__init__\n")
    

============Running inside TestChild class:

running metaclass FixSigMeta.__new__

local variables are:

{'__class__': <class '__main__.FixSigMeta'>,
 'bases': (<class '__main__.AutoInit'>, <class '__main__.TestParent'>),
 'cls': <class '__main__.PrePostInitMeta'>,
 'dict': {'__init__': <function TestChild.__init__>,
          '__module__': '__main__',
          '__qualname__': 'TestChild'},
 'name': 'TestChild'}

Using type.__new__ to create a class instance of type: <class '__main__.TestChild'>
"TestChild's type should and is: <class '__main__.PrePostInitMeta'>"

TestChild's inheritance tree: (<class '__main__.TestChild'>, <class '__main__.AutoInit'>, <class '__main__.TestParent'>, <class 'object'>)
In other words, TestChild's bases are: (<class '__main__.AutoInit'>, <class '__main__.TestParent'>)


does TestChild have attr `__signature__`:False
<Signature (*args, **kwargs)>
if TestChild has its own __init__, then update with __init__'s signature

now, TestChild.__signature__ becomes

In [ ]:
t = TestChild()
test_eq(t.h, 10) # h=10 is initialized in the parent class
test_eq(t.k, 12)

Running PrePostInitMeta.__call__

local variables:
{'args': (), 'cls': <class '__main__.TestChild'>, 'kwargs': {}}

Task 1 running res = TestChild.__new__(cls) to create an obj instance of TestChild: <__main__.TestChild object>

Task 2: 
if TestChild object is an instance of TestChild and TestChild has '__pre_init__' attr, then run TestChild.__pre_init__ on the object.

====Running inside AutoInit.__pre_init__: 

Now, object.__init__ is called to initialize, so that subclasses of AutoInit         not need to run this line of code.

Task 3: 
if TestChild object is an instance of TestChild, then run TestChild.__init__ on the object.
====running TestChild.__init__


Task 4: 
if TestChild object is an instance of TestChild and TestChild has '__post_init__' attr, then run TestChild.__post_init__ on the object.

finally, return the initialized object of class TestChild
====================end of PrePostInitMeta.__call__


how-class: how-metaclass: a great answer on [class](https://stackoverflow.com/questions/100003/what-are-metaclasses-in-python/6581949#6581949) and on [metaclass](https://hyp.is/Z7kBshlQEe25MUNPsjGP0w/stackoverflow.com/questions/100003/what-are-metaclasses-in-python)

## 

## 

#|hide
## Export

In [ ]:
#|hide
from nbdev import nbdev_export
nbdev_export()

#|hide
## Sending to Obs

In [ ]:
#|hide
!jupytext --to md /Users/Natsume/Documents/debuggable/fastcore/00_delegates.ipynb
!mv /Users/Natsume/Documents/debuggable/fastcore/00_delegates.md \
/Users/Natsume/Documents/divefastai/Debuggable/jupytext/fastcore/

[jupytext] Reading /Users/Natsume/Documents/debuggable/fastcore/00_delegates.ipynb in format ipynb
[jupytext] Writing /Users/Natsume/Documents/debuggable/fastcore/00_delegates.md


In [ ]:
!jupyter nbconvert --config /Users/Natsume/Documents/mynbcfg.py --to markdown \
--output-dir /Users/Natsume/Documents/divefastai/Debuggable/nbconvert

[NbConvertApp] Converting notebook /Users/Natsume/Documents/debuggable/index.ipynb to markdown
[NbConvertApp] Writing 7680 bytes to /Users/Natsume/Documents/divefastai/Debuggable/nbconvert/index.md
[NbConvertApp] Converting notebook /Users/Natsume/Documents/debuggable/utils.ipynb to markdown
[NbConvertApp] Writing 14526 bytes to /Users/Natsume/Documents/divefastai/Debuggable/nbconvert/utils.md
[NbConvertApp] Converting notebook /Users/Natsume/Documents/debuggable/fastcore/delegates_keep.ipynb to markdown
[NbConvertApp] Writing 11469 bytes to /Users/Natsume/Documents/divefastai/Debuggable/nbconvert/delegates_keep.md
[NbConvertApp] Converting notebook /Users/Natsume/Documents/debuggable/fastcore/00_delegates.ipynb to markdown
[NbConvertApp] Writing 69707 bytes to /Users/Natsume/Documents/divefastai/Debuggable/nbconvert/00_delegates.md
[NbConvertApp] Converting notebook /Users/Natsume/Documents/debuggable/fastcore/classes_metaclasses.ipynb to markdown
[NbConvertApp] Writing 27393 bytes to